# Task 1 - Exploratory Data Analysis (EDA)
- This notebook aims to present the findings of my analysis, and also a detailed walkthrough of the steps taken to derive the results.

Outline of Steps

a. [Data Pre-processing](#data-pre-processing)

- [Preparing Database Files](#1-preparing-database-db-files)
- [Data Cleaning](#2-data-cleaning)

b. [Features Selection](#features-selection)

# Data Pre-processing
Purpose:
- Data pre-processing is a crucial step in the data analysis and machine learning pipeline as it prepare and clean raw data in a way that makes it suitable for further analysis or modelling.
## 1. Preparing database ('.db') files

#### Use `sqlite3` library to prepare database ('.db') files

In [18]:
# Import required libraries
import sqlite3
import pandas as pd

#### Define a 'get database connection' function for reusability and readability

In [19]:
# Define a function that returns database connection object

def get_db_connection(url):
    conn = sqlite3.connect(url)        # points database connection object to conn
    conn.row_factory = sqlite3.Row     # set return rows as dictionaries instead of tuples
    return conn

#### Load **cruise_pre.db** and **cruise_post.db** into Python environment
- Dataset are read into a list of dictionaries, with each dictionary representing an individual row of data

- This is an intermediary step before converting to DataFrame

In [20]:
# Read cruise_pre.db and cruise_post.db into list of dictionaries

# Define required variables to store data from db
cruise_pre_data = []
cruise_post_data = []

# Define variable to store name of the tables
table_name = ['cruise_pre','cruise_post']

# Iterate through the 2 provided data sets: cruise_pre.db and cruise_post.db
for table in table_name:

    # Create connection to database using defined function
    conn = get_db_connection('data/'+ table + '.db')
    # Execute query to fetch all rows and columns
    data = conn.execute('SELECT * FROM {}'.format(table)).fetchall()

    # Append row to respective variable store
    if table == 'cruise_pre':
        for row in data:
            cruise_pre_data.append(dict(row))
    else:
        for row in data:
            cruise_post_data.append(dict(row))

# close database connection after completion of task
conn.close()

# Output: cruise_pre_data and cruise_post_data now contains list of dictionaries

#### Convert list of dict to Pandas DataFrame
- Convert to Pandas DataFrame objects as it unlocks a wide variety of simple and clean data analytics libraries

In [21]:
# Read list of dictionaries into Pandas Dataframe
# Set 'index' column as index
cruise_post_df = pd.DataFrame(data=cruise_post_data).set_index('index')
cruise_pre_df = pd.DataFrame(data=cruise_pre_data).set_index('index')

## 2. Data Cleaning

#### Dataset 1: Pre-cruise Customer Survey
Description
- This is a pre-purchase survey conducted to give ShipSail insights into what ameneties/services their customers prefer, or what is considered important to them for an enjoyable cruise journey.

Observation 1: **inconsistent scale across amenities**

- For example, `Onboard Wifi Service` has a categorical scale while `Ease of Online booking` has a numerical scale
- Implications: 
    1. inconsistent format makes data plotting across these variables difficult
    2. machine learning algorithms require numeric input variables
- Data handling suggestion:
    1. encode all categorical scale into corresponding numerical scale of 1.0 to 5.0

Observation 2: **missing values, or NaN fields**

- For example, `guest 133745` has 'NaN' for `Port Check-in Service`
- Implications:
    1. Similar to Observation 1
- Data handling suggestion:
    1. Removal can be an option, but substantial rows will have to be removed for this set
    2. Impute 'NaN' into median value of that column to keep more data points and minimize large spreads or inaccuracies


In [22]:
# Display part of cruise pre dataset
cruise_pre_df.head()

,Gender,Date of Birth,Source of Traffic,Onboard Wifi Service,Embarkation/Disembarkation time convenient,Ease of Online booking,Gate location,Logging,Onboard Dining Service,Online Check-in,Cabin Comfort,Onboard Entertainment,Cabin service,Baggage handling,Port Check-in Service,Onboard Service,Cleanliness,Ext_Intcode
index,,,,,,,,,,,,,,,,,,
0,None,05/10/1973,Direct - Company Website,A little important,3.0,5.0,3.0,01/01/2023 0:00,Very important,2.0,2.0,A little important,2.0,2.0,4.0,2.0,3.0,LB446RWOOZI
1,Female,None,Indirect - Social Media,Not at all important,4.0,1.0,NaN,01/01/2023 0:01,Very important,NaN,4.0,None,2.0,3.0,4.0,4.0,4.0,LB138HKBECM
2,Female,22/07/1998,Indirect - Search Engine,None,3.0,0.0,5.0,01/01/2023 0:02,None,NaN,5.0,Extremely important,2.0,1.0,2.0,3.0,NaN,BL713UHBAAN
3,Female,01/05/1970,Direct - Company Website,Very important,4.0,4.0,4.0,01/01/2023 0:05,Somewhat important,4.0,4.0,None,NaN,NaN,3.0,2.0,4.0,LB243DMKCFL
4,Male,07/01/1960,Direct - Company Website,Somewhat important,4.0,2.0,NaN,01/01/2023 0:06,Not at all important,2.0,NaN,Not at all important,3.0,NaN,5.0,2.0,NaN,LB218CFLOBS


#### Encoding Pre-cruise Survey
Encode categorical labels to numerical label because:
1. inconsistent format makes data plotting across these variables difficult
2. machine learning algorithms require numeric input variables

3 Scale Columns are categorically labelled: 
1. `Onboard Wifi Service`
2. `Onboard Entertainment`
3. `Onboard Dining Service`

In addition, these Guest Detail Column are categorically labelled as well:
1. `Gender`


Look for unique values within each column first.

In [23]:
# Summarise the unique values in this column
scale_summary = pd.concat([cruise_pre_df.groupby('Onboard Wifi Service').size(),
                        cruise_pre_df.groupby('Onboard Entertainment').size(),
                        cruise_pre_df.groupby('Onboard Dining Service').size()],
                        axis=1,
                        keys=['Wifi Service Count', 'Entertainment Count', 'Dining Service Count'])
scale_summary

,Wifi Service Count,Entertainment Count,Dining Service Count
A little important,29383,20009,24763
Extremely important,13031,28625,25149
Not at all important,20236,14205,14468
Somewhat important,29101,21627,25037
Very important,22503,33367,27520


In [24]:
# Based on the unique labels, there are no misspelling or outliers to be handled
# Note: NaN values are present but not displayed here

# Define a string to value map to help encode these columns
scale_map = {'Not at all important':1.0,
                'A little important':2.0,
                'Somewhat important':3.0,
                'Very important': 4.0,
                'Extremely important':5.0}

# Store columns to encode
encode_columns = ['Onboard Wifi Service', 'Onboard Entertainment', 'Onboard Dining Service']

# Iterate encoding for each column
for column in encode_columns:
    cruise_pre_df[column] = cruise_pre_df[column].map(scale_map)

For Gender, encode `Male` to 0 and `Female` to 1:

In [25]:
# Define a string to value map to encode gender column
gender_map = {'Male':0.0, 'Female':1.0}

# Encode 'Gender' column based on their gender
cruise_pre_df['Gender'] = cruise_pre_df['Gender'].map(gender_map)

In [26]:
cruise_pre_df.groupby('Source of Traffic').size()

Source of Traffic
Direct - Company Website    54591
Direct - Email Marketing    54645
Indirect - Search Engine    12288
Indirect - Social Media     12222
dtype: int64

In [27]:
# Define a string to value map to encode source of traffic column, based on the unique labels
traffic_map = {'Direct - Company Website':0.0,
               'Direct - Email Marketing':1.0,
               'Indirect - Search Engine':2.0,
               'Indirect - Social Media':3.0}

# Encode 'Source of Traffic' column
cruise_pre_df['Source of Traffic'] = cruise_pre_df['Source of Traffic'].map(traffic_map)

In [30]:
# Show part of the resulting dataframe
cruise_pre_df.head()

,Gender,Date of Birth,Source of Traffic,Onboard Wifi Service,Embarkation/Disembarkation time convenient,Ease of Online booking,Gate location,Logging,Onboard Dining Service,Online Check-in,Cabin Comfort,Onboard Entertainment,Cabin service,Baggage handling,Port Check-in Service,Onboard Service,Cleanliness,Ext_Intcode
index,,,,,,,,,,,,,,,,,,
0,NaN,05/10/1973,0.0,2.0,3.0,5.0,3.0,01/01/2023 0:00,4.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,3.0,LB446RWOOZI
1,1.0,None,3.0,1.0,4.0,1.0,NaN,01/01/2023 0:01,4.0,NaN,4.0,NaN,2.0,3.0,4.0,4.0,4.0,LB138HKBECM
2,1.0,22/07/1998,2.0,NaN,3.0,0.0,5.0,01/01/2023 0:02,NaN,NaN,5.0,5.0,2.0,1.0,2.0,3.0,NaN,BL713UHBAAN
3,1.0,01/05/1970,0.0,4.0,4.0,4.0,4.0,01/01/2023 0:05,3.0,4.0,4.0,NaN,NaN,NaN,3.0,2.0,4.0,LB243DMKCFL
4,0.0,07/01/1960,0.0,3.0,4.0,2.0,NaN,01/01/2023 0:06,1.0,2.0,NaN,1.0,3.0,NaN,5.0,2.0,NaN,LB218CFLOBS


Results after encoding step:

**Scales, Gender and Source of Traffic that were in categorical scale are now in numerical scale.**

#### Handling Missing Values for Pre-cruise Survey
Many column contains missing or NaN values, this step aims to either impute or remove them.

**Scale Columns**
- For scale columns (where guests indicate their level of preference), we can replace NaN values with that column's most occuring importance, aka the `mode`.
- This approach leverages on available data to estimate the missing scales.
- Allows more data points to be kept, while also not skewing the data if extreme default values are used (e.g 0)
- `Mean` can be used too but since scales are whole numbers, `mode` is slightly more suited.

**Gender Column**
- Employ the same approach, replacing NaN fields with `mode` of `Gender` column

In [39]:
# Define a list of column names that belongs to guest's scale rating
scale_columns = ['Onboard Wifi Service',
                 'Embarkation/Disembarkation time convenient',
                 'Ease of Online booking',
                 'Gate location',
                 'Onboard Dining Service',
                 'Online Check-in',
                 'Cabin Comfort',
                 'Onboard Entertainment',
                 'Cabin service',
                 'Baggage handling',
                 'Port Check-in Service',
                 'Onboard Service',
                 'Cleanliness']
# Iterate through each scale column, fill NaN fields with the mode of that column
for column in scale_columns:
    cruise_pre_df[column].fillna(cruise_pre_df[column].mode()[0], inplace=True)

# Similarly for Gender, fill NaN gender field with mode of the Gender column
cruise_pre_df['Gender'].fillna(cruise_pre_df['Gender'].mode()[0], inplace=True)


**Resulting `cruise_pre_df` so far:**
- All scales are in proper numerical label, and missing values are filled via `mode` imputation
- Gender was assigned binary label (`Male` = 0, `Female` = 1), and missing values are filled via `mode` imputation
- Source of Traffic was ordinally labelled and missing values filled via `mode` imputation
    - `'Direct - Company Website':0.0`
    - `'Direct - Email Marketing':1.0`
    - `'Indirect - Search Engine':2.0`
    - `'Indirect - Social Media':3.0`

In [41]:
cruise_pre_df.head()

,Gender,Date of Birth,Source of Traffic,Onboard Wifi Service,Embarkation/Disembarkation time convenient,Ease of Online booking,Gate location,Logging,Onboard Dining Service,Online Check-in,Cabin Comfort,Onboard Entertainment,Cabin service,Baggage handling,Port Check-in Service,Onboard Service,Cleanliness,Ext_Intcode
index,,,,,,,,,,,,,,,,,,
0,1.0,05/10/1973,0.0,2.0,3.0,5.0,3.0,01/01/2023 0:00,4.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,3.0,LB446RWOOZI
1,1.0,None,3.0,1.0,4.0,1.0,3.0,01/01/2023 0:01,4.0,4.0,4.0,4.0,2.0,3.0,4.0,4.0,4.0,LB138HKBECM
2,1.0,22/07/1998,2.0,2.0,3.0,0.0,5.0,01/01/2023 0:02,4.0,4.0,5.0,5.0,2.0,1.0,2.0,3.0,4.0,BL713UHBAAN
3,1.0,01/05/1970,0.0,4.0,4.0,4.0,4.0,01/01/2023 0:05,3.0,4.0,4.0,4.0,4.0,4.0,3.0,2.0,4.0,LB243DMKCFL
4,0.0,07/01/1960,0.0,3.0,4.0,2.0,3.0,01/01/2023 0:06,1.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,4.0,LB218CFLOBS


#### Dataset 2: Post-cruise Customer Data
Description
- This is a post-trip data collected by ShipSail to provide insights into the services or products that customers chosen in reality.

Observation 1: 

- For example,
- Implications: 

- Data cleaning process:

Observation 2: 

- For example,
- Implications:

- Data cleaning process:



In [29]:
# Display part of cruise post dataset
cruise_post_df.head(10)

,Cruise Name,Ticket Type,Cruise Distance,Ext_Intcode,WiFi,Dining,Entertainment
index,,,,,,,
0,Blastoise,None,3567 KM,LB446RWOOZI,1.0,1,1.0
1,Blastoise,Deluxe,672 KM,LB138HKBECM,NaN,0,1.0
2,IAPRAS,Deluxe,1167 KM,BL713UHBAAN,NaN,0,0.0
3,Lapras,Deluxe,280 KM,LB243DMKCFL,NaN,0,1.0
4,Lapras,Standard,1145 Miles,LB218CFLOBS,NaN,1,NaN
5,Lapras,Standard,1145 Miles,LB218CFLOBS,NaN,1,NaN
6,Lapras,Standard,607 KM,BL115AWWXPU,NaN,1,NaN
7,Blastoise,None,528 KM,LB313ANTIPC,NaN,1,NaN
8,Blastoise,Deluxe,None,BL537GQMWKI,NaN,0,1.0


#### Data Cleaning and Imputation (Post-cruise Data)

# Features Selection